In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import torch
import torch.nn as nn

# from train_evaluate import NMR_prediction

import torch.nn.functional as F
import dgl
import dgl.nn as dglnn
import torch.nn as nn
from dgl import AddSelfLoop

from model_2d.NMR_gcn import NMR_GCN

from train_evaluate_2d import NMR_prediction

from preprocess.create_node_embedding.node_embeddings_godess import create_node_embeddings

from preprocess.build_adjaency_matrix.create_adjaency_matrix_godess import build_adjacency_matrix

from preprocess.create_graph.create_graph_data_godess import create_graph

# from dgl import save_graphs, load_graphs
# from tqdm import tqdm
from dgl import save_graphs, load_graphs

##### A Baseline model for GODESS using random forest

We follow the same way of loading and testing the data as 2D GNN model, please refer 2D_GODESS.ipynb.

Here, we encode the atom-level and monosaccharide-level features (fingerprints) as node embeddings.

However, we use atom features as input to the random forest model, but ignore the atom connections.

In [2]:
# data directory
data_dir = 'godess/data/'

# define the lengtht of the embeddings of each feature
atom_name_dim=32

bound_orig_dim=4

atom_type_dim=4

ab_dim=8

dl_dim=8

pf_dim=8

mono_accurate_dim=16

mono_simple_dim=32

me_dim=4

ser_dim=4

s_dim=8

ac_dim=8

gc_dim=4

In [3]:
C = create_node_embeddings(data_dir = data_dir)
C.write_all_embeddings(atom_name_dim = atom_name_dim, bound_orig_dim = bound_orig_dim, 
                           atom_type_dim = atom_type_dim, ab_dim = ab_dim, dl_dim = dl_dim,
                           pf_dim = pf_dim, mono_accurate_dim = mono_accurate_dim, 
                           mono_simple_dim = mono_simple_dim, me_dim=me_dim, 
                          ser_dim=ser_dim, s_dim=s_dim, ac_dim=ac_dim, gc_dim=gc_dim)

100%|██████████████████████████████████████| 2310/2310 [00:05<00:00, 403.98it/s]


In [4]:
threshold_carbon=1.65

threshold_hydrogen=1.18

threshold_general=1.5

threshold_interaction=5.0

In [5]:
B = build_adjacency_matrix(labeled_pdb_dir = data_dir,
                           threshold_carbon=threshold_carbon,
                           threshold_hydrogen=threshold_hydrogen, 
                           threshold_general=threshold_general, 
                           threshold_interaction=threshold_interaction)
B.calculate_all_matrix()

100%|███████████████████████████████████████| 2310/2310 [01:29<00:00, 25.69it/s]


In [6]:
num_test = 462
Create = create_graph(data_dir=data_dir, num_test=num_test)
g, test_index = Create.create_all_graph()

--------------------------loading NMR Graph-------------------------------


100%|███████████████████████████████████████| 2310/2310 [01:14<00:00, 31.01it/s]


In [15]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# g = g.to(device)

features = g.ndata['z']

labels = g.ndata['y']

# masks = g.ndata['train_carbon_mask'], g.ndata['test_carbon_mask']

masks = g.ndata['train_hydrogen_mask'], g.ndata['test_hydrogen_mask']

In [16]:
train_x = features.numpy()[masks[0]]
test_x = features.numpy()[masks[1]]

train_y = labels.numpy()[masks[0]]
test_y = labels.numpy()[masks[1]]

In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

regr = RandomForestRegressor(random_state = 0)

regr.fit(train_x, train_y)

RandomForestRegressor(random_state=0)

In [22]:
pred_y = regr.predict(test_x)

In [23]:
import numpy as np

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [24]:
rmse(pred_y, test_y)

0.13213551225380524